In [1]:
import numpy
import numpy as np

import bokeh
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import Figure, output_notebook, show
from bokeh.layouts import column, row

output_notebook()

Loading BokehJS ...

In [2]:
x = 30*np.random.random((10,))
y = 30*np.random.random((10,))

xyp = np.random.random((10,6,))
xyp.sort(axis=1)
xyp = np.exp(1j * 2*np.pi * xyp)
xyp = np.array([np.real(xyp), np.imag(xyp)])
xyp = xyp.transpose(1, 2, 0).copy()

x = x[:, None] + xyp[..., 0]
y = y[:, None] + xyp[..., 1]

t = np.random.random((10,5))
tr = np.repeat([np.arange(t.shape[1])], t.shape[0], axis=0)

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = Figure(plot_width=400, plot_height=400, tools=["tap"], title="Select Here")
p1.patches('x', 'y', source=s1, alpha=0.6)

s2a = ColumnDataSource(data=dict(t=t, tr=tr))
s2b = ColumnDataSource(data=dict(ts=[], trs=[]))
p2 = Figure(plot_width=400, plot_height=400, x_range=(tr[0][0],tr[0][-1]), y_range=(t.min(),t.max()),
            tools="", title="Watch Here")
p2.line('trs', 'ts', source=s2b, alpha=0.6)

s1.callback = CustomJS(args=dict(s2a=s2a, s2b=s2b), code="""
    var inds = cb_obj.get('selected')['1d'].indices;

    var d1 = cb_obj.get('data');
    var d2a = s2a.get('data');
    var d2b = s2b.get('data');

    d2b['trs'] = [];
    d2b['ts'] = [];
    if (inds.length > 0) {
        for (i = 0; i < d2a['tr'][0].length; i++) {
            d2b['trs'].push(d2a['tr'][inds[0]][i]);
            d2b['ts'].push(d2a['t'][inds[0]][i]);
        }
    }
    s2b.trigger('change');
""")

layout = row(p1, p2)

show(layout)